In [1]:
# Generate data for the Darcy flow problem
# Input: a - the permeability "ground truth"
# output: u - the flow through the permeability
# data generated:
# 2 datasets of 200 'Devito' pairs, to compare with the ground truth, with resolutions of:
# 128, 64

In [ ]:
pip install torch

In [ ]:
pip install tensorflow

In [4]:
import torch
import math
from devito import div, grad, Eq, Operator, TimeFunction, Function, solve, Grid
import numpy as np
from random_fields import GaussianRF
from timeit import default_timer
import scipy.io

2022-08-05 09:51:02.135699: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-05 09:51:02.135757: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
import os

In [6]:
os.environ['DEVITO_LANGUAGE'] = 'openmp'

os.environ['OMP_SCHEDULE'] = 'STATIC'

os.environ['OMP_PROC_BIND'] = 'true'

os.environ['OMP_PLACES'] = 'cores'

In [7]:

'''
function to generate 'u' from 'a' using Devito
parameters 
-----------------
perm: tensor of size (1, s, s)
    this is "a"
f: tensor of size (s, s)
    the forcing function f(x) = 1
s: int
    the resolution
 '''
def darcy_flow_2d(perm, f, s):
    
    # a(x) is the coefficients
    # f is the forcing function
    # give the a and f1 functions the same data as inputs perm and f
    for i in range(s):
        f1.data[i] = f[i]
        a.data[i] = perm[i]
    
    # call operator for the nth psuedo-timestep
    if s == 128:
        n = 7500
    else:
        n = 3750
    
    op(time=n)
   
    return torch. from_numpy(u.data[0])

In [8]:
# this is used to generate "devito" data results, to compare to the FNO results and the ground truth

test_128 = scipy.io.loadmat('df_test_128.mat') 
test_64 = scipy.io.loadmat('df_test_64.mat') 

# for each test mat file:
for testcase in [test_128, test_64]:
   
    # number of grid points on [0,1]^2 
    s = testcase['coeff'].shape[1]
    print("this is s", s)
    
    # Number of solutions to generate
    N = 200

    # Forcing function, f(x) = 1 
    f = np.ones((s, s))

    # create s x s grid with spacing 1
    grid = Grid(shape=(s, s), extent=(1.0,1.0))

    # create function on grid
    # space order of 2 to enable 2nd derivative
    # TimeFunction for u can be used despite the lack of a time-dependence. This is done for psuedotime
    u = TimeFunction(name='u', grid=grid, space_order=2)
    a = Function(name='a', grid=grid, space_order=2)
    f1 = Function(name='f1', grid=grid, space_order=2)

    # define 2D Darcy flow equation
    equation_u = Eq(-a*u.laplace,f1)

    # Let SymPy solve for the central stencil point
    stencil = solve(equation_u, u)

    # Let our stencil populate the buffer `u.forward`
    update = Eq(u.forward, stencil)

    # Boundary Conds
    x, y = grid.dimensions
    t = grid.stepping_dim
    nx = s
    ny = s
    bc = [Eq(u[t+1, 0, y],u[t+1, 1,y])]  # du/dx = 0 for x=0.
    bc += [Eq(u[t+1,nx-1, y],u[t+1,nx-2, y])]  # du/dx = 0 for x=1.
    bc += [Eq(u[t+1, x, 0],u[t+1,x ,1])]  # du/dx = 0 at y=0
    bc += [Eq(u[t+1, x, ny-1],u[t+1, x, ny-2])]  # du/dx=0 for y=1
    # u=0 for all sides
    bc = [Eq(u[t+1, x, 0], 0.)]
    bc += [Eq(u[t+1, x, ny-1], 0.)]
    bc += [Eq(u[t+1, 0, y], 0.)]
    bc += [Eq(u[t+1, nx-1, y], 0.)]

    op = Operator([update] + bc)


    # all the inputs
    coeff = np.zeros(shape=(N, s, s))
    #all the solutions
    sol = torch.zeros(N, s, s)
     
    c = 0
    t0 = default_timer()
    
    # for each input generate a new solution
    for each_input in testcase['coeff']:
        
        w1 = each_input
        
        #Solve df
        output = darcy_flow_2d(w1, f, s)
        
        coeff[c:(c+1),...] = w1
        sol[c:(c+1),...] = output
        
        c += 1
        t1 = default_timer()
        print("sample no. :", c, "Time taken:", t1-t0)
    
    scipy.io.savemat('df_devito_{}.mat'.format(s), mdict={'coeff': coeff, 'sol': sol.cpu().numpy()})

Equation is not affine w.r.t the target, falling back to standardsympy.solve that may be slow


this is s 128


Operator `Kernel` ran in 0.09 s
Operator `Kernel` ran in 0.10 s


sample no. : 1 Time taken: 0.20345066394656897
sample no. : 2 Time taken: 0.3389128502458334


Operator `Kernel` ran in 0.09 s
Operator `Kernel` ran in 0.09 s


...

sample no. : 181 Time taken: 6.2211163099855185
sample no. : 182 Time taken: 6.258174368180335
sample no. : 183 Time taken: 6.290170380845666
sample no. : 184 Time taken: 6.324164914898574
sample no. : 185 Time taken: 6.35714312735945
sample no. : 186 Time taken: 6.390649099834263
sample no. : 187 Time taken: 6.421764826402068


Operator `Kernel` ran in 0.02 s
Operator `Kernel` ran in 0.02 s
Operator `Kernel` ran in 0.02 s
Operator `Kernel` ran in 0.02 s
Operator `Kernel` ran in 0.02 s
Operator `Kernel` ran in 0.02 s
Operator `Kernel` ran in 0.02 s


sample no. : 188 Time taken: 6.453635789453983
sample no. : 189 Time taken: 6.48692726995796
sample no. : 190 Time taken: 6.519029404968023
sample no. : 191 Time taken: 6.552779438905418
sample no. : 192 Time taken: 6.586091274395585
sample no. : 193 Time taken: 6.621518775820732


Operator `Kernel` ran in 0.02 s
Operator `Kernel` ran in 0.02 s
Operator `Kernel` ran in 0.02 s
Operator `Kernel` ran in 0.02 s
Operator `Kernel` ran in 0.02 s
Operator `Kernel` ran in 0.02 s


sample no. : 194 Time taken: 6.655564463697374
sample no. : 195 Time taken: 6.69232617225498
sample no. : 196 Time taken: 6.724692068062723
sample no. : 197 Time taken: 6.755761465989053
sample no. : 198 Time taken: 6.786722993478179
sample no. : 199 Time taken: 6.8170746164396405
sample no. : 200 Time taken: 6.850807716138661
